In [2]:
from metar import Metar
from datetime import datetime, timedelta
from metar import Metar
from util.utils import *
import pandas as pd
import re


In [5]:

def interpretaMetaR(metarString):

    if("METAF" in metarString):
        metarString = metarString.replace('METAF', 'METAR')
        metarString = metarString.replace('\n', '')


    station = None
    type = None
    hora_formated = None
    temperature = None
    dew_point = None
    windDirection = None
    windVelocity = None
    visibility = None
    pressure = None
    weather = None
    sky = None
    skyFeet= None

    try: 

        obs = Metar.Metar(metarString[:-1])

        for item in obs.string().split('\n'):
            if("station" in item):
                station = item.split(" ")[1]
            if("type" in item):
                type = " ".join(item.split(" ")[1:])
            if("time" in item):
                timeZ = " ".join(item.split(" ")[1:])
                data_obj = datetime.strptime(timeZ, "%a %b %d %H:%M:%S %Y")
                nova_data_obj = data_obj - timedelta(hours=3)
                hora_formated = nova_data_obj.strftime("%a %b %d %H:%M:%S %Y")
            if("temperature" in item):
                temperature = item.split(" ")[1]
            if("dew point" in item):
                dew_point = item.split(" ")[2]
            if("wind" in item):
                wind = (item.split(" ")[1:])
                windDirection = wind[0]
                windVelocity = wind[2]
            if("visibility" in item):
                visibility = item.split(" ")[1]
            if("pressure" in item):
                pressure = item.split(" ")[1]
            if("weather" in item):
                weather = " ".join(item.split(" ")[1:])
            if("sky" in item):
                skyString = " ".join(item.split(" ")[1:])
                if(' at ' in skyString):
                    sky = ''.join(skyString.split(' at ')[0])
                else:
                    sky = ''.join(skyString.split(',')[0])
                
                allFeets = re.findall(r"(\d+)",skyString)
                if len(allFeets) > 0:
                    skyFeet = allFeets[0]
    except:
        pass

    return station, type, hora_formated, temperature,dew_point,windDirection,windVelocity,visibility,pressure,weather,sky,skyFeet

def direction_to_degrees(df):
    direction_degrees = {
    'N': 0,
    'NNE': 22.5,
    'NE': 45,
    'ENE': 67.5,
    'E': 90,
    'ESE': 112.5,
    'SE': 135,
    'SSE': 157.5,
    'S': 180,
    'SSW': 202.5,
    'SW': 225,
    'WSW': 247.5,
    'W': 270,
    'WNW': 292.5,
    'NW': 315,
    'NNW': 337.5,
    'calm': 0,
    'variable': 0,
    }
    try:
        df['metar_windDirection'] = df['metar_windDirection'].map(direction_degrees)
    except:
        df['metaf_windDirection'] = df['metaf_windDirection'].map(direction_degrees)

    return df

def defineIndex(df):
    df = df.sort_values(by='hora_formatted')
    df.set_index('hora_formatted', inplace=True)
    df.index = pd.to_datetime(df.index, format='%A, %B %d, %Y %H:%M:%S')
    return df

def to_dummies(df):
    df_Dummies = pd.get_dummies(df['sky'])
    df_final = pd.concat([df, df_Dummies], axis=1)

    return df_final

def sort_df(df):
    df = df.drop_duplicates()
    df = df.dropna()
    df = df.sort_values(by=['aero','hora_formatted'])

    return df
def calculate_average(series):
    result = []
    for i in range(len(series) - 1):
        current_value = series.iloc[i]
        next_value = series.iloc[i + 1]
        average = (current_value + next_value) / 2
        result.append(average)
    return pd.Series(result, index=series.index[:-1])

def pipemediaTempoMetar(df, colunas):
    listSeries = []
    non_numeric_columns = []

    for coluna in colunas:
        if pd.api.types.is_numeric_dtype(df[coluna]):
            df[coluna] = pd.to_numeric(df[coluna], errors='coerce')
            df_resampled_temp = df[coluna].resample('30T').mean()
            df_resampled_temp_filled = df_resampled_temp.fillna(method='ffill')
            tempSeries = calculate_average(df_resampled_temp_filled)
            listSeries.append(tempSeries)
        else:
            non_numeric_columns.append(coluna)

    dataframe_resultante = pd.concat(listSeries, axis=1)

    # Criar um novo DataFrame para armazenar as colunas não numéricas
    df_nao_numericas = df[non_numeric_columns]

    # Redefinir os índices dos DataFrames antes da concatenação
    dataframe_resultante.reset_index(drop=True, inplace=True)
    df_nao_numericas.reset_index(drop=True, inplace=True)

    # Concatenar o DataFrame resultante com o DataFrame das colunas não numéricas
    resultado_final = pd.concat([dataframe_resultante, df_nao_numericas], axis=1)

    return resultado_final

def read_metar(df,tabela):
    if (tabela == "metar"):
        resultado_df =  df['metar'].apply(interpretaMetaR)
        resultado_df =  pd.DataFrame(resultado_df.tolist(),columns=['metar_station', 'metar_type', 'metar_hora_formated', 'metar_station','metar_dew_point','metar_windDirection','metar_windVelocity','metar_visibility','metar_pressure','metar_weather','metar_sky','metar_skyFeet'] )
    else:
        resultado_df =  df['metaf'].apply(interpretaMetaR)
        resultado_df =  pd.DataFrame(resultado_df.tolist(),columns=['metaf_station', 'metaf_type', 'metaf_hora_formated', 'metaf_station','metaf_dew_point','metaf_windDirection','metaf_windVelocity','metaf_visibility','metaf_pressure','metaf_weather','metaf_sky','metaf_skyFeet'] )
    df_bronze = pd.concat([df, resultado_df], axis=1)
    df_bronze = direction_to_degrees(df_bronze)
    # df_bronze = sort_df(df_bronze)
    # df_bronze = df_bronze.reset_index(drop=True)
    # df_bronze['hora_formatted'] = pd.to_datetime(df_bronze['hora_formatted'])
    # df_bronze.set_index(['aero','hora_formatted'], inplace=True)
    return df_bronze
    df_media = pipemediaTempoMetar(df_bronze,['hora','aero','dew_point', 'windDirection', 'windVelocity', 'visibility','pressure', 'weather', 'sky', 'skyFeet'])
    df_no_columns = df_bronze.drop(['dew_point', 'windDirection', 'windVelocity', 'visibility','pressure', 'weather', 'sky', 'skyFeet'],axis=1)
    return df_media,df_bronze



In [6]:
df1 = pd.read_csv("../data/processed_data/merged_bronze_01.csv")
df2 = pd.read_csv("../data/processed_data/merged_bronze_02.csv")
df3 = pd.read_csv("../data/processed_data/merged_bronze_03.csv")

In [7]:
df = pd.concat([df1, df2,df3], axis=0)
df.shape

(79468, 25)

In [14]:
df.columns

Index(['Unnamed: 0', 'flightid', 'origem', 'destino', 'dt_dep', 'dt_arr',
       'dt_dep_formatted', 'dt_arr_formatted', 'hora_real', 'nova_cabeceira',
       'antiga_cabeceira', 'aero_real', 'hora_formatted_real', 'hora_prev',
       'troca', 'aero_prev', 'hora_formatted_prev', 'hora_metar', 'metar',
       'aero_metar', 'hora_formatted_metar', 'hora_metaf', 'metaf',
       'aero_metaf', 'hora_formatted_metaf'],
      dtype='object')

In [19]:

df = df.reset_index()
df.head(5)

,index,flightid,origem,destino,dt_dep,dt_arr,dt_dep_formatted,dt_arr_formatted,hora_real,nova_cabeceira,...,aero_prev,hora_formatted_prev,hora_metar,metar,aero_metar,hora_formatted_metar,hora_metaf,metaf,aero_metaf,hora_formatted_metaf
0,1,e7820d2da06dc2f494350c9b60a12213,SBKP,SBGL,1654100598000,1654103254000,"Wednesday, June 01, 2022 13:23:18","Wednesday, June 01, 2022 14:07:34",1654100271000,33,...,SBGL,"Wednesday, June 01, 2022 14:00:00",1654099200000,METAR SBKP 011600Z 02006KT 9999 FEW020 BKN100 ...,SBKP,"Wednesday, June 01, 2022 13:00:00",1654102800000,METAF SBGL 011700Z 17005KT 9999 SCT033 ...,SBGL,"Wednesday, June 01, 2022 14:00:00"
1,2,ea6b1a0829b22c8f32b3afd4329ed3e9,SBKP,SBGL,1654208990000,1654211428000,"Thursday, June 02, 2022 19:29:50","Thursday, June 02, 2022 20:10:28",1654209698000,15,...,SBGL,"Thursday, June 02, 2022 20:00:00",1654207200000,METAR SBKP 022200Z 12011KT CAVOK 19/14 Q1017=,SBKP,"Thursday, June 02, 2022 19:00:00",1654210800000,METAF SBGL 022300Z 23006KT 5000 BR OVC012...,SBGL,"Thursday, June 02, 2022 20:00:00"
2,3,68fbb3bfc9aab75d6ec44eb311c8f053,SBKP,SBGL,1654208224000,1654210696000,"Thursday, June 02, 2022 19:17:04","Thursday, June 02, 2022 19:58:16",1654209698000,15,...,SBGL,"Thursday, June 02, 2022 20:00:00",1654207200000,METAR SBKP 022200Z 12011KT CAVOK 19/14 Q1017=,SBKP,"Thursday, June 02, 2022 19:00:00",1654210800000,METAF SBGL 022300Z 23006KT 5000 BR OVC012...,SBGL,"Thursday, June 02, 2022 20:00:00"
3,4,8c523b08cd8cbcf6a75301a1f95d844a,SBKP,SBGL,1654552500000,1654556105000,"Monday, June 06, 2022 18:55:00","Monday, June 06, 2022 19:55:05",1654550762000,33,...,SBGL,"Monday, June 06, 2022 20:00:00",1654552800000,METAR SBKP 062200Z 02002KT CAVOK 22/15 Q1017=,SBKP,"Monday, June 06, 2022 19:00:00",1654556400000,METAF SBGL 062300Z 07002KT CAVOK 23/17 Q1015=\n,SBGL,"Monday, June 06, 2022 20:00:00"
4,5,1398a677760469fb0194afc0bde237d1,SBKP,SBGL,1654554806000,1654557517000,"Monday, June 06, 2022 19:33:26","Monday, June 06, 2022 20:18:37",1654556306000,15,...,SBGL,"Monday, June 06, 2022 20:00:00",1654556400000,METAR SBKP 062300Z 10003KT CAVOK 21/14 Q1018=,SBKP,"Monday, June 06, 2022 20:00:00",1654556400000,METAF SBGL 062300Z 07002KT CAVOK 23/17 Q1015=\n,SBGL,"Monday, June 06, 2022 20:00:00"


In [20]:
df = read_metar(df,"metar")
df = read_metar(df,"metaf")

In [22]:
df.columns

Index(['index', 'flightid', 'origem', 'destino', 'dt_dep', 'dt_arr',
       'dt_dep_formatted', 'dt_arr_formatted', 'hora_real', 'nova_cabeceira',
       'antiga_cabeceira', 'aero_real', 'hora_formatted_real', 'hora_prev',
       'troca', 'aero_prev', 'hora_formatted_prev', 'hora_metar', 'metar',
       'aero_metar', 'hora_formatted_metar', 'hora_metaf', 'metaf',
       'aero_metaf', 'hora_formatted_metaf', 'metar_station', 'metar_type',
       'metar_hora_formated', 'metar_station', 'metar_dew_point',
       'metar_windDirection', 'metar_windVelocity', 'metar_visibility',
       'metar_pressure', 'metar_weather', 'metar_sky', 'metar_skyFeet',
       'metaf_station', 'metaf_type', 'metaf_hora_formated', 'metaf_station',
       'metaf_dew_point', 'metaf_windDirection', 'metaf_windVelocity',
       'metaf_visibility', 'metaf_pressure', 'metaf_weather', 'metaf_sky',
       'metaf_skyFeet'],
      dtype='object')

In [23]:
df[['flightid','dt_dep_formatted', 'dt_arr_formatted','hora_formatted_metar','hora_formatted_metaf']]

,flightid,dt_dep_formatted,dt_arr_formatted,hora_formatted_metar,hora_formatted_metaf
0,e7820d2da06dc2f494350c9b60a12213,"Wednesday, June 01, 2022 13:23:18","Wednesday, June 01, 2022 14:07:34","Wednesday, June 01, 2022 13:00:00","Wednesday, June 01, 2022 14:00:00"
1,ea6b1a0829b22c8f32b3afd4329ed3e9,"Thursday, June 02, 2022 19:29:50","Thursday, June 02, 2022 20:10:28","Thursday, June 02, 2022 19:00:00","Thursday, June 02, 2022 20:00:00"
2,68fbb3bfc9aab75d6ec44eb311c8f053,"Thursday, June 02, 2022 19:17:04","Thursday, June 02, 2022 19:58:16","Thursday, June 02, 2022 19:00:00","Thursday, June 02, 2022 20:00:00"
3,8c523b08cd8cbcf6a75301a1f95d844a,"Monday, June 06, 2022 18:55:00","Monday, June 06, 2022 19:55:05","Monday, June 06, 2022 19:00:00","Monday, June 06, 2022 20:00:00"
4,1398a677760469fb0194afc0bde237d1,"Monday, June 06, 2022 19:33:26","Monday, June 06, 2022 20:18:37","Monday, June 06, 2022 20:00:00","Monday, June 06, 2022 20:00:00"
...,...,...,...,...,...
79462,b23e81538959936ec302da8871581de1,"Monday, May 01, 2023 16:13:32","Monday, May 01, 2023 16:54:58","Monday, May 01, 2023 16:00:00","Monday, May 01, 2023 17:00:00"
79463,e4f15bae6b848825b0b6deb92485e8e7,"Monday, May 01, 2023 16:21:00","Monday, May 01, 2023 17:00:59","Monday, May 01, 2023 16:00:00","Monday, May 01, 2023 17:00:00"
79464,425066a19f1d74998744667b3cacfd15,"Monday, May 01, 2023 17:09:06","Monday, May 01, 2023 17:49:49","Monday, May 01, 2023 17:00:00","Monday, May 01, 2023 18:00:00"
79465,8dff390d5351442fe4bd5c51b1618c16,"Monday, May 01, 2023 17:20:32","Monday, May 01, 2023 18:03:40","Monday, May 01, 2023 17:00:00","Monday, May 01, 2023 18:00:00"


In [24]:
df = df.drop_duplicates()

df.shape

(79467, 49)

In [39]:
df.to_csv("../data/processed_data/merged_plate.csv")